<a href="https://colab.research.google.com/github/martin-kosiik/scraping-court-cases/blob/master/extract_foreign_courts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np

In [ ]:
!pip install razdel

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [19]:
final_dataset = pd.read_csv('drive/My Drive/data/final_dataset_2.csv', encoding="UTF-8")

arbitrage_rulings_df = pd.read_csv("drive/My Drive/data/arbitrage_rulings.csv", encoding="UTF-8")
arbitrage_rulings_df['ruling_date'] = arbitrage_rulings_df.case_id.str.extract('__(\d{8})( *\(\d\))*$')[0]
arbitrage_rulings_df['ruling_date'] = pd.to_datetime(arbitrage_rulings_df['ruling_date'], format='%Y%m%d')
arbitrage_rulings_df['starting_year'] = arbitrage_rulings_df.case_id.str.extract('(\d{4})__\d{8} *(\(\d\))*$')[0]

arbitrage_rulings_df['case_number'] = arbitrage_rulings_df['court_id'] + '-' + arbitrage_rulings_df['case_spec_digit'].astype(int).astype(str) + '/' + \
                                        arbitrage_rulings_df['starting_year'].astype(int).astype(str)


key_diff = set(arbitrage_rulings_df['case_number']).intersection(final_dataset['Номер дела'])
where_diff = arbitrage_rulings_df['case_number'].isin(key_diff)
arbitrage_rulings_df = arbitrage_rulings_df[where_diff]


# Remove the the NEWPAGE token and the page number
arbitrage_rulings_df["proc_text"] = arbitrage_rulings_df.text.str.replace("NEWPAGE \n\d", "",
                                                                       regex=True)

arbitrage_rulings_df["proc_text"] = arbitrage_rulings_df.proc_text.str.replace("NEWPAGE", "")

In [ ]:
from  razdel  import sentenize , tokenize
#from  navec  import   Navec 
def make_tokens(text):
  all_tokens_in_text  = []
  for sent in sentenize(text):
      tokens = [_.text  for  _  in  tokenize(sent.text)]
      all_tokens_in_text.append(tokens)
  return all_tokens_in_text


tokenized_rulings = [make_tokens(ruling) for ruling in arbitrage_rulings_df["proc_text"]]

In [5]:
!pip install stanza
!pip install fuzzymatcher

     |████████████████████████████████| 235kB 7.0MB/s 
     |████████████████████████████████| 51kB 3.8MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144789 sha256=b7020bb50502be10a7d9139baa8f5190bb48f969597cc3b20650cc8f272c96cd
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
  Created wheel for metaphone: filename=Metaphone-0.6-cp36-none-any.whl size=13907 sha256=673da351eb65e6d61ee2c71911f5f18bbdedafe37a630c5511c3a91669905219
  Stored in directory: /root/.cache/pip/wheels/4e/7c/f7/162d726fc83491ef23c7a0b989005024eb83a1408c96f32eaf
Successfully built python-Levenshtein metaphone


In [ ]:
!pip install spacy-stanza


In [ ]:
!pip install pymorphy2==0.8

     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 7.1MB 20.9MB/s 


In [ ]:
import stanza
#from spacy_stanza import StanzaLanguage
#from spacy import displacy
stanza.download('ru')

2020-11-24 08:00:21 INFO: Downloading default packages for language: ru (Russian)...
2020-11-24 08:01:20 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
# https://stanfordnlp.github.io/stanza/tokenize.html
snlp = stanza.Pipeline(lang="ru", processors='tokenize,lemma,ner', tokenize_pretokenized=True)
#nlp = StanzaLanguage(snlp)
#nlp = 

2020-11-24 10:06:41 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| lemma     | syntagrus |
| ner       | wikiner   |

2020-11-24 10:06:41 INFO: Use device: gpu
2020-11-24 10:06:41 INFO: Loading: tokenize
2020-11-24 10:06:41 INFO: Loading: lemma
2020-11-24 10:06:41 INFO: Loading: ner
2020-11-24 10:06:53 INFO: Done loading processors!


In [ ]:
%%timeit
stanza_docs_list = [snlp(text) for text in arbitrage_rulings_df["proc_text"][:2]]



1 loop, best of 3: 501 ms per loop


In [ ]:
arbitrage_rulings_df["proc_text"]

3        \n    \n \nАРБИТРАЖНЫЙ СУД ИРКУТСКОЙ ОБЛАСТИ ...
4        \n \n \nАрбитражный суд Липецкой области \n39...
5        \n \n \n \nАрбитражный суд Мурманской области...
6                                                     ...
7        \n \n                      \n                ...
                              ...                        
3364    1/2019-151565(1) \n \n \nАрбитражный суд Яросл...
3370    59/2019-46141(2) \n \nАРБИТРАЖНЫЙ СУД ЯРОСЛАВС...
3374    151/2019-65742(2) \n \nАРБИТРАЖНЫЙ СУД ЯРОСЛАВ...
3375    151/2019-68068(2) \n \nАРБИТРАЖНЫЙ СУД ЯРОСЛАВ...
3376    88/2019-68098(2) \n \nАРБИТРАЖНЫЙ СУД ЯРОСЛАВС...
Name: proc_text, Length: 2168, dtype: object

In [ ]:
stanza_docs_list = [snlp(text) for text in arbitrage_rulings_df["proc_text"]]

#text = arbitrage_rulings_df["proc_text"][2]
#doc = snlp(text)

In [ ]:
import pickle 
file_to_wr = open('drive/My Drive/data/stanza/stanza_docs_list.obj', 'wb')
pickle.dump(stanza_docs_list, file_to_wr)


In [6]:
import pickle
import stanza
stanza_docs_list = pickle.load( open( "drive/My Drive/data/stanza/stanza_docs_list.obj", "rb" ) )


In [ ]:
import re

captured_match = re.search('суд', list(doc.ents)[0].text, flags=re.IGNORECASE)

courts = [entity.text for entity in list(doc.ents) if re.search('суд', entity.text, flags=re.IGNORECASE) is not None]
print(captured_match.string)
courts
#list(doc.ents)

ТРИНАДЦАТЫЙ АРБИТРАЖНЫЙ АПЕЛЛЯЦИОННЫЙ СУД


['ТРИНАДЦАТЫЙ АРБИТРАЖНЫЙ АПЕЛЛЯЦИОННЫЙ СУД',
 'Тринадцатый арбитражный апелляционный суд',
 'Арбитражного  суда  Санкт-Петербурга',
 'Арбитражный  суд  Санкт-Петербурга']

In [ ]:
list(doc.ents)[12].text

'Арбитражного  суда  Санкт-Петербурга'

In [7]:
import re
list_to_filter = ['арбитражный суд', 'суда', 'суд', 'решения суда', 'арбитражного суда', 'арбитражным судом', 'указанное решение суда',
                  'окружного суда', 'решение иностранного суда', 'апк рф, суд', 'апк рф, арбитражный суд', 'межрайонного суда',
                  'судом', 'верховный суд', 'арбитражный суд г.', 'верховного суда']
def find_courts(nlp_doc):
  ent_list = [ent.text for ent in nlp_doc.entities]
  courts = [entity for entity in ent_list if re.search('(^суд\s|^суда\s|^судом\s|court|\sсуд\s|суда\s|судом\s|суд$|судом$|суда$)', entity, flags=re.IGNORECASE) is not None]
  courts = [court.replace('\n','').lower() for court in courts]
  courts = [court for court in courts if court not in list_to_filter]
  courts = [court for court in courts if re.search('российской|\sрф$|\sмоскв|\sарбитражный апелляционный суд$', court, flags=re.IGNORECASE) is None]
  return list(dict.fromkeys(courts))

In [ ]:
find_courts(stanza_docs_list[9])

['арбитражный суд республики башкортостан',
 'арбитражный суд рб',
 'международного арбитражного суда',
 'арбитражный суд республики',
 'федеральный арбитражный суд']

In [8]:
courts_list = [find_courts(nlp_doc) for nlp_doc in stanza_docs_list]

In [20]:
arbitrage_rulings_df.drop(['text'], inplace=True, axis=1)

In [21]:
courts_list.__len__()
arbitrage_rulings_df['courts_list'] = courts_list
#arbitrage_rulings_df.join(final_dataset, how='left', on = )
arbitrage_rulings_df = pd.merge(arbitrage_rulings_df, final_dataset[['Номер дела', 'plaintiff_country_1', 'plaintiff_country_2', 'defendant_country_1', 'defendant_country_2']], how = 'left', left_on='case_number', right_on='Номер дела')

In [23]:
arbitrage_rulings_df.to_excel('drive/My Drive/data/courts_list.xlsx', encoding='utf-8', index=False)

In [ ]:
# межрайонного , Херсон, минска
#хозяйственным, суда комрат республики молдова
# полтавской области, арбитражного суда туркменистана
# житомирской
# земельного суда мюнхена
# межрайонного экономического суда
np.array([len(court) for court in courts_list]).mean()



3.5276752767527677

In [ ]:
def pot_ukr_court(courts):
  output = []
  for court in courts:
    if re.search('хозяйствен', '' if court ==[] else court, flags=re.IGNORECASE) is None:
      pass
    else:
      output.append(court)
  return output

pot_ukr_court(courts_list[5])
pot_ukr_court([])



potential_ukr_courts = [pot_ukr_court(court) for court in courts_list]

In [ ]:
potential_ukr_courts
#courts_list[1]

[[],
 ['хозяйственного суда',
  'хозяйственного суда г. киева украины',
  'хозяйственного суда г. киева.',
  'хозяйственного суда г. киева'],
 [],
 ['хозяйственного суда'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['хозяйственного суда', 'хозяйственного суда минской области'],
 [],
 ['хозяйственного суда автономной республики',
  'хозяйственного суда автономной республики крым',
  'хозяйственного суда автономной'],
 [],
 [],
 [],
 [],
 ['хозяйственного суда г. минска'],
 ['хозяйственного суда г. минска'],
 [],
 [],
 ['хозяйственного суда'],
 ['хозяйственного суда',
  'хозяйственного суда города минска',
  'хозяйственный суд города минска.'],
 ['хозяйственного суда'],
 [],
 [],
 [],
 [],
 [],
 ['хозяйственного суда'],
 [],
 ['хозяйственного суда'],
 ['хозяйственного суда'],
 [],
 ['хозяйственного суда'],
 ['хозяйственного суда',
  'приказов хозяйственного суда гомельской области',
  'хозяйственного суда гомельской',
  'приказов хозяйственного суда'],
 [],
 [],
 ['хозяйств

In [ ]:
courts_list

[['арбитражный суд иркутской области',
  'решения международного коммерческого арбитражного суда',
  'международного коммерческого арбитражного суда',
  'решение международного коммерческого арбитражного суда'],
 ['хозяйственного суда',
  'хозяйственного суда г. киева украины',
  'хозяйственного суда г. киева.',
  'хозяйственного суда г. киева'],
 ['арбитражного суда мурманской области',
  'карагандинского областного суда',
  'федеральный арбитражный суд северо-западного округа'],
 ['арбитражный суд мурманской области',
  'арбитражного суда мурманской области',
  'хозяйственного суда',
  'федеральный арбитражный суд северо-западного округа'],
 ['арбитражный суд свердловской области',
  'атырауского областного суда республики',
  'арбитражный суд свердловской',
  'атырауского областного суда республики казахстан',
  'атырауского областного суда'],
 [],
 ['арбитражный суд астраханской области',
  'экономического суда',
  'решением экономического суда по',
  'экономическим судом азербайдж

In [ ]:
russian_courts_df = pd.read_excel("drive/My Drive/data/russian_courts.xlsx")
russian_courts_df['russian_courts'] = russian_courts_df.russian_courts.str.replace('\s\(.+\)', '')

In [ ]:
import fuzzymatcher
fuzzymatcher.fuzzy_left_join(pd.DataFrame(courts, columns=['entity']), russian_courts_df, left_on = "entity", right_on = "russian_courts")

,best_match_score,__id_left,__id_right,entity,russian_courts
0,0.177106,0_left,16_right,ТРИНАДЦАТЫЙ АРБИТРАЖНЫЙ АПЕЛЛЯЦИОННЫЙ СУД,Тринадцатый арбитражный апелляционный суд
1,0.177106,1_left,16_right,Тринадцатый арбитражный апелляционный суд,Тринадцатый арбитражный апелляционный суд
2,-0.144556,2_left,96_right,Арбитражного суда Санкт-Петербурга,Арбитражный суд Санкт-Петербурга и Ленинградск...
5,0.007809,3_left,96_right,Арбитражный суд Санкт-Петербурга,Арбитражный суд Санкт-Петербурга и Ленинградск...
6,0.177106,4_left,16_right,Тринадцатый арбитражный апелляционный суд,Тринадцатый арбитражный апелляционный суд
7,-0.244900,5_left,96_right,ПОСТАНОВИЛ: Решение Арбитражного суда Санкт-П...,Арбитражный суд Санкт-Петербурга и Ленинградск...
10,0.217344,6_left,8_right,Федеральный арбитражный суд Северо-Западного...,Арбитражный суд Северо-Западного округа
